# Python Module

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext
SparkContext.setSystemProperty('spark.executor.memory', '2g')
sc = SparkContext("local", "App Name")

In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
from pyspark.sql import SQLContext, Row

In [6]:
from pyspark.sql.functions import col

# 파일 불러오기

In [7]:
spark = SparkSession.builder.getOrCreate()

In [8]:
data = spark.read.csv("./Crimes_Chicago.csv", header=True)

In [9]:
chicago = data.alias("chicago")

# 결측지 제거

In [10]:
chicago = data.na.drop(subset =["Longitude"])

# 년도별 카운트

In [11]:
chicago.createOrReplaceTempView("CHICAGO")

In [12]:
year_count = spark.sql("SELECT Year, count(Year) AS Count FROM CHICAGO GROUP BY Year ORDER BY Year")

# 월/일/시간 추가

In [13]:
# 복사
chicago_md = chicago

In [14]:
chicago_md = chicago_md.withColumn('Month', chicago_md['Date'].substr(1,2))
chicago_md = chicago_md.withColumn('Day', chicago_md['Date'].substr(4,2))

In [15]:
chicago_md = chicago_md.withColumn('Time', chicago_md['Date'].substr(12,15))

In [16]:
chicago_md.createOrReplaceTempView("CHICAGO_MD")

* Column 명칭 변경

In [17]:
chicago_md_rename = chicago_md.select('ID',col("Case Number").alias("Case_Number"), 'Date','Block','IUCR',
                                      col("Primary Type").alias("Primary_Type"),'Description',
                                      col("Location Description").alias("Location_Description"), 
                                      'Arrest','Domestic','Beat','District','Ward',
                                      col("Community Area").alias("Community_Area"),
                                      col("FBI Code").alias("FBI_Code"),
                                     col("X Coordinate").alias("X_Coordinate"),col("Y Coordinate").alias("Y_Coordinate"),
                                      col("Updated On").alias("Updated_On"),'Latitude','Longitude','Location',
                                     col("Historical Wards 2003-2015").alias("Historical_Wards_20032015"),
                                     col("Zip Codes").alias("Zip_Codes"),col("Community Areas").alias("Community_Areas"),
                                     col("Census Tracts").alias("Census_Tracts"), 'Wards',
                                      col("Boundaries - ZIP Codes").alias("Boundaries_ZIPCodes"),
                                     col("Police Districts").alias("Police_Districts"), col("Police Beats").alias("Police_Beats"),
                                      'Year','Month', 'Day','Time')

In [18]:
chicago_md_rename.createOrReplaceTempView("CHICAGO_MD_RE")

# 년도별 데이터

In [19]:
year_lst = list(year_count.select('Year').toPandas()['Year'])

In [20]:
yearlst = []
for i in range(len(year_lst)):
#     print(year_lst[i])
    tmp = spark.sql("SELECT * FROM CHICAGO_MD_RE WHERE Year = {}".format(year_lst[i]))
    yearlst.append(tmp)

* crime 데이터 합치기

In [21]:
crime1617 = yearlst[15].union(yearlst[16])